In [16]:
import subprocess
import time

import pexpect

In [33]:
import subprocess
import shlex
import time

# SLURM allocation parameters
partition = "RM-shared"
num_nodes = 1
ntasks_per_node = 4
time_limit = "00:30:00"  # HH:MM:SS

# Construct the salloc command
salloc_command = (
    f"salloc -J bm --nodes={num_nodes} --partition={partition} "
    f"--ntasks-per-node={ntasks_per_node} --time={time_limit}"
)

salloc_command = f"salloc -J bm --nodes={num_nodes} --partition={partition}"

try:
    print("Requesting SLURM interactive allocation...")

    # Start an salloc process
    salloc_process = subprocess.Popen(
        salloc_command,
        shell=True,
        stdout=subprocess.PIPE,
        # stderr=subprocess.PIPE,
        text=True,
    )

    # Continuously read the stdout and stderr to debug
    print("Monitoring SLURM allocation...")
    while True:
        salloc_process.stdout.flush()
        output = salloc_process.stdout.readline()
        if output == "" and salloc_process.poll() is not None:
            break
        if output:
            print(f"SLURM Output: {output.strip()}")
        time.sleep(0.5)

    # Check if the allocation succeeded
    return_code = salloc_process.poll()
    if return_code != 0:
        stderr_output = salloc_process.stderr.read()
        print(f"SLURM Allocation Failed with code {return_code}")
        print(f"Error Details: {stderr_output.strip()}")
        exit(1)

    print("SLURM allocation successful!")

    # Construct and execute commands in the allocated session
    commands = [
        "module load intelmpi/2021.3.0-intel2021.3.0",
        "which mpirun",
        # f"mpirun -np {num_nodes * ntasks_per_node} ./your_mpi_program",
    ]
    command_script = " && ".join(commands)
    
    print("Executing commands in allocated session...")
    srun_process = subprocess.run(
        command_script,
        shell=True,
        check=True,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
    )

    # Print outputs
    print("Command execution output:")
    print(srun_process.stdout)
    if srun_process.stderr:
        print(f"Command execution errors:\n{srun_process.stderr}")

except subprocess.CalledProcessError as e:
    print(f"Command failed with return code {e.returncode}: {e.cmd}")
    print(f"Error message: {e.stderr}")
except Exception as e:
    print(f"An unexpected error occurred: {e}")


Requesting SLURM interactive allocation...
Monitoring SLURM allocation...
An unexpected error occurred: 'NoneType' object has no attribute 'flush'


salloc: Pending job allocation 28361006
salloc: job 28361006 queued and waiting for resources
salloc: Job has been cancelled
salloc: Job allocation 28361006 has been revoked.
salloc: error: Job submit/allocate failed: Job/step already completing or completed


In [27]:
import subprocess
import shlex

# SLURM allocation parameters
partition = "RM,RM-shared"
num_nodes = 1
ntasks_per_node = 4
# time_limit = "00:30:00"  # HH:MM:SS

# Construct the salloc command
# salloc_command = (
#     f"salloc -J bm --nodes={num_nodes} --partition={partition}"
#     # f"--ntasks-per-node={ntasks_per_node}"
# )

salloc_command = "salloc -J bm --nodes=1 --partition=RM-shared"

with subprocess.Popen(
        salloc_command,
        shell=True,
        stdin=subprocess.PIPE,
        stdout=subprocess.PIPE,
        stderr=subprocess.PIPE,
        text=True,
    ) as salloc_process:

    # Load the MPI module

    # Construct the commands to execute within the interactive session
    commands = [
        "module load intelmpi/2021.3.0-intel2021.3.0",
        "which mpirun",
        # f"mpirun -np {num_nodes * ntasks_per_node} ./your_mpi_program",
    ]
    
    # Join commands into a single script to execute
    command_script = " && ".join(commands) + "\n"
    
    print("Executing commands in interactive session...")
    stdout, stderr = salloc_process.communicate(input=command_script)

print(stdout, stderr)


# try:
#     # Start the salloc process
#     print("Requesting SLURM allocation...")
#     salloc_process = subprocess.run(
#         salloc_command,
#         shell=True,
#         check=True,
#         stdout=subprocess.PIPE,
#         stderr=subprocess.PIPE,
#         text=True,
#     )

#     module_load_command = "module load intelmpi/2021.3.0-intel2021.3.0"
#     # mpi_command = f"mpirun -np {num_nodes * ntasks_per_node} ./your_mpi_program"
#     mpi_command = "which mpirun"
    
#     print("Loading MPI module...")
#     subprocess.run(module_load_command, shell=True, check=True)

#     print("Running MPI job...")
#     mpi_process = subprocess.run(
#         mpi_command,
#         shell=True,
#         check=True,
#         stdout=subprocess.PIPE,
#         stderr=subprocess.PIPE,
#         text=True,
#     )

    
#     # Run an MPI command inside the allocated session
#     # mpi_command = f"mpirun -np {num_nodes*ntasks_per_node} ./your_mpi_program"
#     # mpi_command = "which mpirun"
#     # print("Running MPI job in allocated session...")
#     # stdout, stderr = salloc_process.communicate(input=mpi_command)

#     # # Print outputs
#     # print(stdout)
#     # if stderr:
#     #     print(f"Errors:\n{stderr}")
    
# except Exception as e:
#     print(f"An error occurred: {e}")


Executing commands in interactive session...
/jet/packages/intel/oneapi/mpi/2021.3.0/bin/mpirun
 salloc: Pending job allocation 28360815
salloc: job 28360815 queued and waiting for resources
salloc: job 28360815 has been allocated resources
salloc: Granted job allocation 28360815
srun: error: ioctl(TIOCGWINSZ): Inappropriate ioctl for device
srun: error: Not using a pseudo-terminal, disregarding --pty option
salloc: Relinquishing job allocation 28360815



In [18]:
salloc_cmd = "salloc -J benchmaark --nodes=1 --partition=RM-shared"
# child = pexpect.spawn(salloc_cmd)



# mpirun_cmd = "which mpirun"

# child.sendline(mpirun_cmd)

# # Capture the output of the MPI job
# child.expect(pexpect.EOF, timeout=600)  # Adjust timeout as needed
# output = child.before.decode("utf-8")
# print("MPI Output:", output)

slurm_proc = subprocess.Popen(
    salloc_cmd.split(),
    # stdin=subprocess.PIPE,
    shell=True
    # stdout=subprocess.PIPE,
    # stderr=subprocess.PIPE,
    # text=True
)

time.sleep(25)

mpi_cmd = 'which mpirun'

stdout, stderr = slurm_proc.communicate(input=mpi_cmd, timeout=600)

print(stdout)
print(stderr)


# slurm_proc.stdin.write(f"{mpi_cmd}\n")
# # slurm_proc.stdin.write("exit\n")
# slurm_proc.stdin.flush()

# # Get output
# stdout, stderr = slurm_proc.communicate()

# interactive_session = subprocess.Popen(
#     salloc_cmd.split(),
#     shell=True,
#     # stdin=subprocess.PIPE,
#     # stdout=subprocess.PIPE,
#     # stderr=subprocess.PIPE,
#     # text=True,
# )


# # Wait for the session to be granted
# child.expect("salloc: Nodes .* are ready for job")

# time.sleep(10)

# stdout, stderr = interactive_session.communicate(input='which mpirun', timeout=600)
# print(stdout)
# print(stderr)

salloc: Pending job allocation 28360645
salloc: job 28360645 queued and waiting for resources


KeyboardInterrupt: 

In [9]:
stderr

'salloc: unrecognized option \'--pty\'\nTry "salloc --help" for more information\n'